In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from time import time


In [2]:
# Chargement des données
DataFrame = pd.read_csv("Data_Arbre.csv")

# Préparation des données
DataFrame.loc[DataFrame['fk_arb_etat'] == 'Essouché', 'fk_arb_etat'] = 1
DataFrame.loc[DataFrame['fk_arb_etat'] != 1, 'fk_arb_etat'] = 0
DataFrame['fk_arb_etat'] = DataFrame['fk_arb_etat'].astype('int')

# Sélectionner les colonnes pertinentes
colonnes_numeriques = ['longitude', 'latitude', 'haut_tot', 'haut_tronc', 'tronc_diam', 'age_estim', 'fk_prec_estim', 'clc_nbr_diag']
colonnes_categorieles = ['clc_quartier', 'clc_secteur', 'fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation', 'fk_revetement', 'fk_nomtech', 'feuillage', 'villeca']
DataFrame.loc[DataFrame['remarquable'] == 'Oui', 'remarquable'] = 1
DataFrame.loc[DataFrame['remarquable'] != 1, 'remarquable'] = 0
DataFrame['remarquable'] = DataFrame['remarquable'].astype('int')


In [3]:
# Encodage des variables catégorielles
X = pd.concat([DataFrame[colonnes_numeriques], pd.get_dummies(DataFrame[colonnes_categorieles]), DataFrame['remarquable']], axis=1)
y = DataFrame['fk_arb_etat']

# Standardisation des données
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [4]:
# Appliquer SMOTE sur l'ensemble d'entraînement uniquement
smote = SMOTE(random_state=42, sampling_strategy=0.2)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Entraînement du modèle RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_smote, y_train_smote)

# Importance des caractéristiques
importance = clf.feature_importances_
indices = np.argsort(importance)[::-1]
print("\nImportance des caractéristiques :")
for f in range(X_train_smote.shape[1]):
    print(f"{X.columns[indices[f]]} : {importance[indices[f]]}")

C:\Users\alexm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\alexm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")



Importance des caractéristiques :
haut_tronc : 0.07065932020756124
longitude : 0.05997828826926328
fk_prec_estim : 0.0593221880897146
age_estim : 0.05865434254471422
latitude : 0.053552659138133264
clc_nbr_diag : 0.046368773936973896
tronc_diam : 0.04383401958423219
clc_secteur_Parc des Champs-Elysées : 0.04060886607138821
haut_tot : 0.03793442782277682
fk_situation_Groupe : 0.03230426113763347
fk_situation_Alignement : 0.027940651025509466
fk_nomtech_SORAUC : 0.026944832874896116
clc_quartier_Quartier Remicourt : 0.02369303054256468
fk_pied_gazon : 0.019823961670794647
clc_secteur_Cimetière Sud : 0.01925011651426298
fk_situation_Isolé : 0.01554673445686732
fk_nomtech_BETPEN : 0.0135413656312829
clc_quartier_Quartier du faubourg d'Isle : 0.013385442250298479
fk_port_semi libre : 0.012184252967115228
fk_nomtech_ACEPLA : 0.01155715320843206
clc_secteur_Auberge de jeunesse : 0.011382072786198634
fk_pied_Terre : 0.010977633656163716
villeca_CASQ : 0.01091909378740915
clc_quartier_Quartier

In [5]:
# Utiliser les caractéristiques les plus importantes pour entraîner un nouveau modèle
X_train_selected = X_train_smote[:, indices[:20]]
clf_selected = RandomForestClassifier(random_state=42)
clf_selected.fit(X_train_selected, y_train_smote)
X_test_selected = X_test[:, indices[:20]]

# Prédiction
y_pred = clf_selected.predict(X_test_selected)

# Évaluation du modèle
print("\nAccuracy : ", accuracy_score(y_test, y_pred))
print("\nClassification Report :")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix :")
print(confusion_matrix(y_test, y_pred, normalize='true'))
print("\naccuracy Score :")
print(accuracy_score(y_test, y_pred))


Accuracy :  0.9793072424651372

Classification Report :
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2181
           1       0.42      0.26      0.32        42

    accuracy                           0.98      2223
   macro avg       0.70      0.63      0.66      2223
weighted avg       0.98      0.98      0.98      2223


Confusion Matrix :
[[0.99312242 0.00687758]
 [0.73809524 0.26190476]]

accuracy Score :
0.9793072424651372
